In [349]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [350]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [351]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 150
LOSS_FUNCTION = nn.MSELoss()

In [352]:
y_loss = []
X_epochs = []

In [353]:
def data_workflow(input_data, target_value):
    X = input_data.drop([target_value], axis=1)
    y = input_data[target_value]

    #print(X)
    #print(y)

    X = X.to_numpy()
    y = y.to_numpy()

    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    data = TensorDataset(X, y)

    train_ds, test_ds = train_test_split(data, test_size=0.2, random_state=25)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

    print(f"training data: {train_dl}\n test data: {test_dl}")
    return train_dl, test_dl
    



In [354]:
csv_filepath = "./final_harvest_data.csv"

data = pd.read_csv(csv_filepath)
DATA = data.drop(['date', 'index', 'plant_id', 'tray_id', 'row', 'column'], axis=1)

DATA

,LFW_g,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
0,1.30,0.078,31.95,4.3,5.2,5.8,1211,1456.0,0.831731,189.923880,49,42,346,16,45.684494,40.965988,92.878510,0.442608
1,2.10,0.148,44.10,5.3,5.7,5.5,1412,1556.5,0.907164,181.338094,56,41,370,20,53.368065,37.484673,96.255425,0.711802
2,3.36,0.196,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
3,3.07,0.184,66.98,5.8,7.6,7.5,1601,1787.0,0.895915,203.480229,44,59,395,21,55.633369,38.548523,15.127802,0.721031
4,3.17,0.187,68.74,7.2,8.0,7.1,1919,2372.0,0.809022,263.421354,62,58,454,19,60.012882,48.875011,55.774792,0.580292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,57.96,3.010,726.46,18.5,18.5,13.0,19084,22974.0,0.830678,767.938160,193,167,1367,33,190.580887,137.733093,62.138748,0.691160
161,81.46,3.880,1001.79,21.0,21.5,14.8,18373,21556.5,0.852318,661.612260,164,189,1317,29,176.040924,141.141724,22.056726,0.597653
162,140.84,6.380,1707.14,23.0,22.5,16.9,23374,26050.0,0.897274,683.754395,186,194,1427,31,191.379974,164.496170,175.959305,0.511091
163,108.17,5.250,1364.18,31.4,20.5,16.6,23457,25678.5,0.913488,671.754395,198,179,1364,31,194.815384,159.870819,122.656914,0.571464


In [355]:
LFW_train, LFW_test = data_workflow(DATA, 'LFW_g')

training data: <torch.utils.data.dataloader.DataLoader object at 0x7fda993e7850>
 test data: <torch.utils.data.dataloader.DataLoader object at 0x7fda98583b80>


In [356]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(17, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            
        ) 

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits.double()
        



In [357]:
model = DeepNeuralNetwork()
model = model.double()
print(model)

DeepNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [358]:
def train_loop(dataset, model, loss_function, optimizer):
    model.train()
    for (X, y) in dataset:
        #X, y = X.to('cuda'), y.to('cuda')
        y = y.view(-1,1) 
        
        model.zero_grad()

        prediction = model(X)
        loss = loss_function(prediction, y)
        y_loss.append(loss)
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss}")
        #print(f"X: {X} \n Y: {y}")

In [359]:
def test_loop(dataset, model, loss_function):
    size = len(dataset.dataset)
    num_batches = len(dataset)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for (X, y) in dataset:
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            

    test_loss /= num_batches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [360]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
count = 0
for t in range(EPOCHS):
    count += 1
    X_epochs.append(count)
    print(f"Epoch {t+1}\n------------------------------------")
    train_loop(LFW_train, model, LOSS_FUNCTION, optimizer)
    #test_loop(LFW_test, model, LOSS_FUNCTION)
print("Training complete!")

Epoch 1
------------------------------------
Loss: 2128628.6442596056
Loss: 3344283.242787921
Loss: 1876042.0218082503
Loss: 2505528.118704681
Loss: 605922.7124002817
Epoch 2
------------------------------------
Loss: 1345445.2202109117
Loss: 1320925.017825121
Loss: 1292900.220800491
Loss: 1224167.7165712984
Loss: 14840.256612612891
Epoch 3
------------------------------------
Loss: 561852.2925914266
Loss: 780710.6953442498
Loss: 318508.4231002876
Loss: 516225.93033495604
Loss: 280522.3209397014
Epoch 4
------------------------------------
Loss: 124873.41802440376
Loss: 108193.2403235146
Loss: 378419.2112969244
Loss: 36476.37311694876
Loss: 20456.736679310536
Epoch 5
------------------------------------
Loss: 82792.22240693751
Loss: 8143.474557220041
Loss: 40455.37385904088
Loss: 1460.624178276316
Loss: 3355.8164907013975
Epoch 6
------------------------------------
Loss: 22031.46329721473
Loss: 21390.073081576556
Loss: 19518.485282023677
Loss: 26644.020134839197
Loss: 53780.3568165824